In [1]:
from MITgcmutils.utils import writebin
import numpy as np
from numpy import cos, pi
import xarray as xr

nx = 168    # gridpoints in x
ny = 30    # gridpoints in y
nz = 10    # gridpoints in z

dx = 1     # grid spacing in x (degrees longitude)
dy = 1     # grid spacing in y (degrees latitude)
x0 = 120     # origin in x,y for ocean domain
y0 = -15    # (i.e. southwestern corner of ocean domain)
x1 = x0 + (nx-1)*dx    # origin in x,y for ocean domain
y1 = y0 + (ny-1)*dy    # (i.e. southwestern corner of ocean domain)
opath = '../input/'
ipath = '/mnt/d/project/IAVNNG/Data/'
fbath = ipath + 'GLO-MFC_001_030_mask_bathy.nc'
fclim = ipath + 'cmems_climatology_mon.nc'
fwind = ipath + 'era5_tau_mon.nc'
inc = dx*12
winc = dx*4

## 生成海表面风应力场 （Agrid）

In [2]:
isclim = True
yrs = 10
ds = xr.open_dataset(fwind)

nt = ds['valid_time']
taux = ds['ewss'][:12*yrs].loc[:,y0:y1:-1*winc,x0:x1:winc] 
tauy = ds['nsss'][:12*yrs].loc[:,y0:y1:-1*winc,x0:x1:winc]

taux = taux.values/86400  # era5提供的风应力是一天的积分，需要除以86400转换为风应力
tauy = tauy.values/86400  # 同上
if isclim:
    taux = taux.reshape(-1,12,ny,nx).mean(0)
    tauy = tauy.reshape(-1,12,ny,nx).mean(0)
print(taux.shape, tauy.shape)
# # 保存为MITgcm驱动所需的二进制文件
writebin(opath+'taux.bin',taux)
writebin(opath+'tauy.bin',tauy)

(12, 30, 168) (12, 30, 168)


## 生成EXF Relax边界（SSH SSS Agrid）

In [3]:
ds = xr.open_dataset(fclim)
thetao = ds['thetao'][:,0].loc[:,y0:y1:inc,x0:x1:inc]
so = ds['so'][:,0].loc[:,y0:y1:inc,x0:x1:inc]
theta = thetao.values
so = so.values
print(theta.shape)
writebin(opath+'sst_clim.bin',theta)
writebin(opath+'sss_clim.bin',so)


(12, 30, 168)


## 生成温盐流初始场

In [4]:
from scipy import ndimage
def fill_nan_nearest_2d(arr3d):
    """
    对 4D 数组 (time, depth, lat, lon)，在最后两个维度上进行最近邻插值填补 NaN。
    """
    filled = arr3d.copy()
    Z, Y, X = arr3d.shape
    for z in range(Z):
        slice2d = arr3d[z, :, :]
        if np.isnan(slice2d).any():
            mask = np.isnan(slice2d)
            # 获取最近非 NaN 元素的索引
            idx = ndimage.distance_transform_edt(mask, return_distances=False, return_indices=True)
            filled[z, :, :] = slice2d[tuple(idx)]
    return filled
import matplotlib.pyplot as plt
ds = xr.open_dataset(fclim)
dep = ds['depth'].values
dr = np.zeros(dep.shape)
lev = 0
for i in range(dep.size):
    dr[i] = 2*(dep[i]-lev)
    lev += dr[i]
drr = dr.reshape(nz,-1).sum(-1)
dr = dr[:,None,None]
drr = drr[:,None,None]

ds = xr.open_dataset(fclim)
thetao = ds['thetao'][0].loc[:,y0:y1:inc,x0:x1:inc]
so = ds['so'][0].loc[:,y0:y1:inc,x0:x1:inc]
theta = thetao.values
s = so.values
# 以nearest 在最后两维上对温盐进行nan填补,请补充代码
theta = fill_nan_nearest_2d(theta)
s = fill_nan_nearest_2d(s)

Z, Y, X = theta.shape
theta = (theta*dr).reshape(nz,Z//nz,Y,X).sum(1)/drr
s = (s*dr).reshape(nz,Z//nz,Y,X).sum(1)/drr

print(theta.shape)
writebin(opath+'T.bin',theta)
writebin(opath+'S.bin',s)


(10, 30, 168)


## 生成地形数据

In [5]:
ds = xr.open_dataset(fbath)

hl = ds['deptho'].loc[y0:y1:inc,x0:x1:inc]
hr = ds['deptho'].loc[y0:y1:inc,x0-360:x1-360:inc]
display(hl,hr)
h = np.hstack((hl.values,hr.values))
print(h.shape)
writebin(opath+'bathy.bin',h)

<xarray.DataArray 'deptho' (latitude: 30, longitude: 60)> Size: 7kB
[1800 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float32 120B -15.0 -14.0 -13.0 ... 12.0 13.0 14.0
  * longitude  (longitude) float32 240B 120.0 121.0 122.0 ... 177.0 178.0 179.0
Attributes:
    long_name:      Bathymetry
    standard_name:  sea_floor_depth_below_geoid
    units:          m

<xarray.DataArray 'deptho' (latitude: 30, longitude: 108)> Size: 13kB
[3240 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float32 120B -15.0 -14.0 -13.0 ... 12.0 13.0 14.0
  * longitude  (longitude) float32 432B -180.0 -179.0 -178.0 ... -74.0 -73.0
Attributes:
    long_name:      Bathymetry
    standard_name:  sea_floor_depth_below_geoid
    units:          m

(30, 168)
